In [1]:
from bootstrap import bs_means_diff, bs_stdofmean
from decadal_stats import collect_data, compare_periods
from itertools import compress
import numpy as np
import time
import os
import netCDF4 as nc
import datetime
%matplotlib notebook

In [2]:
#Path to RDA results
RDA_path_1 = "/Users/Siwen/Desktop/ferret/bin/meiyu_clean.nc"
RDA_path_2 = "/Users/Siwen/Desktop/ferret/bin/meiyu_2_clean.nc"

years = [[[1951,1979],[1980,2007]],[[1980,1993],[1994,2007]]]
year_tags = ['8007_5179','9407_8093']
periods = [[60,120],[121,160],[161,200],[201,273],[274,320],[1,365]]
period_tags = ['spring','pre-meiyu','meiyu','post-meiyu','fall','all']
latranges = [[0,99],[0,99],[0,99],[27,99],[0,99],[0,99]]
tau = 2

In [14]:
##set up results file
results = {}

In [17]:
for (yrs,yr_tag) in zip(years,year_tags):
    print(yr_tag)
    p_results={}
    
    for period,latrange,p_tag in zip(periods,latranges,period_tags):
        print(p_tag)
        p_results[p_tag] = compare_periods(yrs,period,latrange=latrange,tau=tau)
        
    results[yr_tag] = p_results

8007_5179
spring


/Users/Siwen/RDA/decadal_stats.py:50: RuntimeWarning: invalid value encountered in greater
  lat_1_pass = (lat_1 > latrange[0]) & (lat_1 < latrange[1])
/Users/Siwen/RDA/decadal_stats.py:50: RuntimeWarning: invalid value encountered in less
  lat_1_pass = (lat_1 > latrange[0]) & (lat_1 < latrange[1])
/Users/Siwen/RDA/decadal_stats.py:52: RuntimeWarning: invalid value encountered in greater
  lat_2_pass = (lat_2 > latrange[0]) & (lat_2 < latrange[1])
/Users/Siwen/RDA/decadal_stats.py:52: RuntimeWarning: invalid value encountered in less
  lat_2_pass = (lat_2 > latrange[0]) & (lat_2 < latrange[1])


pre-meiyu
meiyu
post-meiyu
fall
all
9407_8093
spring
pre-meiyu
meiyu
post-meiyu
fall
all


In [ ]:
## save results to NetCDF
savefile = 'Data/decadal_stats.nc'

#create dimensions in NetCDF file
days_dim = outfile.createDimension('time', None)
lat_dim = outfile.createDimension('lat', 22)
dim_tuple = ("time","lat")

## FROM looking at results above, decided that seasonal tau values are the way to go
# hence, choosing lats_pval[2] and so on (where we stored seasonal tau values)

#run through loop and save all differences of interest, as well as p-values
for p1_b, p1_e, p2_b, p2_e, lat_1_diff, lat_diff, lat_1_Z, lat_Z, lat_1_pval, lat_pval in \
                            zip(p1_beg, p1_end, p2_beg, p2_end, lats_1_diff[0], \
                            lats_diff[0], lats_1_Z[0], lats_Z[0], lats_1_pval[0], lats_pval[0]):
    
    #convoluted line of code below guarantees that 2007 will become '07' and not '7'
    yrs_p1 = str(p1_b-1900) + str("{0:0=2d}".format((p1_e-1900)%100))
    yrs_p2 = str(p2_b-1900) + str("{0:0=2d}".format((p2_e-1900)%100))
    years = yrs_p2 + '_' + yrs_p1
    print(years)

    #create variables inside netCDF file
    lats_1_diff_out = outfile.createVariable("lat_1_diff_tau_" + years, 'f8', dim_tuple)
    lats_diff_out = outfile.createVariable("lat_diff_tau_" + years, 'f8', dim_tuple)
    
    lats_1_Z_out = outfile.createVariable("lat_1_Z_tau_" + years, 'f8', dim_tuple)
    lats_Z_out = outfile.createVariable("lat_Z_tau_" + years, 'f8', dim_tuple)
    
    lats_1_pval_out = outfile.createVariable("lat_1_pval_tau_" + years, 'f8', dim_tuple)
    lats_pval_out = outfile.createVariable("lat_pval_tau_" + years, 'f8', dim_tuple)
    
    
    #fill with actual values
    lats_1_diff_out[:] = lat_1_diff
    lats_diff_out[:] = lat_diff
    
    lats_1_Z_out[:] = lat_1_Z
    lats_Z_out[:] = lat_Z
    
    lats_1_pval_out[:] = lat_1_pval
    lats_pval_out[:] = lat_pval

outfile.close()